<a href="https://colab.research.google.com/github/NayeemIMI/Data-Encryption-with-AES-and-RSA-Algorithm/blob/main/AES%2BRSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyCryptodomex
!pip install PyCrypto
!pip install PyCryptodome

import base64
import hashlib
from Cryptodome.Cipher import AES as domeAES
from Cryptodome.Random import get_random_bytes
from Crypto import Random
from Crypto.Cipher import AES as cryptoAES
from random import randrange, getrandbits
from itertools import repeat
from functools import reduce

BLOCK_SIZE = 16
#K="lolololololololo"
key = K.encode()
#print("Original Key: ",K,"\n")
__key__ = hashlib.sha256(key).digest()
#__key__=key
#print("Hashed Key: ",__key__,"\n")

def encrypt(raw):
    BS = cryptoAES.block_size
    pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)
    raw = base64.b64encode(pad(raw).encode('utf8'))
    iv = get_random_bytes(cryptoAES.block_size)
    cipher = cryptoAES.new(key= __key__, mode= cryptoAES.MODE_CFB,iv=iv)
    a= base64.b64encode(iv + cipher.encrypt(raw))
    IV = Random.new().read(BLOCK_SIZE)
    aes = domeAES.new(__key__, domeAES.MODE_CFB, IV)
    b = base64.b64encode(IV + aes.encrypt(a))
    return b

def decrypt(enc):
    passphrase = __key__
    encrypted = base64.b64decode(enc)
    IV = encrypted[:BLOCK_SIZE]
    aes = domeAES.new(passphrase, domeAES.MODE_CFB, IV)
    enc = aes.decrypt(encrypted[BLOCK_SIZE:])
    unpad = lambda s: s[:-ord(s[-1:])]
    enc = base64.b64decode(enc)
    iv = enc[:cryptoAES.block_size]
    cipher = cryptoAES.new(__key__, cryptoAES.MODE_CFB, iv)
    b=  unpad(base64.b64decode(cipher.decrypt(enc[cryptoAES.block_size:])).decode('utf8'))
    return b
#msg = "Nayeem"
msg=input("\n\nEnter plain text: ")
K=input("\nEnter AES key: ")
encrypted_data =encrypt(msg)
#print("Entered Message: ",msg,"\n")
enco=encrypted_data.decode("utf-8")
print("\nCiphertext: ",enco)
print("\n")
decrypted_data = decrypt(encrypted_data)
#print("Decrypted Message: ",decrypted_data)







 
def getPrime(n):
    "Get a n-bit pseudo-random prime"
    def isProbablePrime(n, t = 7):
        def isComposite(a):
            if pow(a, d, n) == 1:
                return False
            for i in range(s):
                if pow(a, 2 ** i * d, n) == n - 1:
                    return False
            return True
     
        assert n > 0
        if n < 3:
            return [False, False, True][n]
        elif not n & 1:
            return False
        else:
            s, d = 0, n - 1
            while not d & 1:
                s += 1
                d >>= 1
        for _ in repeat(None, t):
            if isComposite(randrange(2, n)):
                return False
        return True   
     
    p = getrandbits(n)
    while not isProbablePrime(p):
        p = getrandbits(n)
    return p
 
def inv(p, q):
    """Multiplicative inverse"""
    def xgcd(x, y):
        """Extended Euclidean Algorithm"""
        s1, s0 = 0, 1
        t1, t0 = 1, 0
        while y:
            q = x // y
            x, y = y, x % y
            s1, s0 = s0 - q * s1, s1
            t1, t0 = t0 - q * t1, t1
        return x, s0, t0      
 
    s, t = xgcd(p, q)[0:2]
    assert s == 1
    if t < 0:
        t += q
    return t
 
def genRSA(p, q):
    """Generate public and private keys"""
    phi, mod = (p - 1) * (q - 1), p * q
    if mod < 65537:
        return (3, inv(3, phi), mod)
    else:
        return (65537, inv(65537, phi), mod)    
 
def text2Int(text):
    """Convert a text string into an integer"""
    return reduce(lambda x, y : (x << 8) + y, map(ord, text))
 
def int2Text(number, size):
    """Convert an integer into a text string"""
    text = "".join([chr((number >> j) & 0xff)
                    for j in reversed(range(0, size << 3, 8))])
    return text.lstrip("\x00")
 
def int2List(number, size):
    """Convert an integer into a list of small integers"""
    return [(number >> j) & 0xff
            for j in reversed(range(0, size << 3, 8))]
 
def list2Int(listInt):
    """Convert a list of small integers into an integer"""
    return reduce(lambda x, y : (x << 8) + y, listInt)
 
def modSize(mod):
    """Return length (in bytes) of modulus"""
    modSize = len("{:02x}".format(mod)) // 2
    return modSize
 
def encrypt(ptext, pk, mod):
    """Encrypt message with public key"""
    size = modSize(mod)
    output = []
    while ptext:
        nbytes = min(len(ptext), size - 1)
        aux1 = text2Int(ptext[:nbytes])
        assert aux1 < mod
        aux2 = pow(aux1, pk, mod)
        output += int2List(aux2, size + 2)
        ptext = ptext[size:]
    return output
 
def decrypt(ctext, sk, p, q):
    """Decrypt message with private key
    using the Chinese Remainder Theorem"""
    mod = p * q
    size = modSize(mod)
    output = ""
    while ctext:
        aux3 = list2Int(ctext[:size + 2])
        assert aux3 < mod
        m1 = pow(aux3, sk % (p - 1), p)
        m2 = pow(aux3, sk % (q - 1), q)
        h = (inv(q, p) * (m1 - m2)) % p
        aux4 = m2 + h * q
        output += int2Text(aux4, size)
        ctext = ctext[size + 2:]
    return output
 
if __name__ == "__main__":
 
    from math import log10
    from time import time
 
    def printHexList(intList):
        """Print ciphertext in hex"""
        for index, elem in enumerate(intList):
            if index % 32 == 0:
                print()            
            print("{:02x}".format(elem), end = "")
        print("\n")
 
    def printLargeInteger(number):
        """Print long primes in a formatted way"""
        string = "{:02x}".format(number)
        for j in range(len(string)):
            if j % 64 == 0:
                print()
            print(string[j], end = "")
        print("\n")
 
    def testCase(p, q, msg):
        """Execute test case: generate keys, encrypt message and
           decrypt resulting ciphertext"""
        #print("Key size: {:0d} bits".format(round(log10(p * q) / log10(2))))
        #print("\nPrime #1:", end = "")
        #printLargeInteger(p)
        #print("Prime #2:", end = "")
        #printLargeInteger(q)
        #print("Plaintext:", msg,"\n")
        pk, sk, mod = genRSA(p, q)
        print("\nRSA Public key:",pk)
        print("\nRSA Private key: ",end = "")
        printLargeInteger(sk)
        ctext = encrypt(msg, pk, mod)
        print("RSA Encrypted AES key:", end = "")
        printHexList(ctext)
        ptext = decrypt(ctext, sk, p, q)
        print("RSA Decrypted AES key:", ptext, "\n")
    keysize=int(input("\nEnter the RSA key size in bits:"))
    p1 = getPrime(keysize)
    p2 = getPrime(keysize)
    #msg=input("Enter text: ")
    msg= str(K)
    testCase(p1, p2, msg)
    print("Decrypted Message: ",decrypted_data)



Enter plain text: klo

Enter AES key: kl

Ciphertext:  tES3af0ugAZ6RIHvGaPeGQRm9IZmW9SkIaBWjOsqsJmYNILF1kIERTZDlB0EkL0JDfBizKypzo1LEQuh8ATXhZmoJUDzf2Pw



Enter the RSA key size in bits:35

RSA Public key: 65537

RSA Private key: 
14bbb375b349a5422d

RSA Encrypted AES key:
00000ef2019dec0e3a820d

RSA Decrypted AES key: kl 

Decrypted Message:  klo
